# Multilayer Perceptron - Seoul Bike Sharing Data

Carico moduli necessari:

In [1]:
from tensorflow import keras 
from sklearn.preprocessing import StandardScaler
from utils.pyutils import print_model_scores

import pandas as pd
import numpy as np

Carico dati:

In [2]:
bike_train = pd.read_csv("data/bike_train_dummy.csv")
bike_test = pd.read_csv("data/bike_test_dummy.csv")
bike_valid = pd.read_csv("data/bike_valid_dummy.csv")

In [3]:
print(bike_train.shape, bike_test.shape, bike_valid.shape)

(6307, 33) (876, 33) (1577, 33)


## MLP - all predictors 

Converti i dati in array numpy:

In [3]:
columns = bike_train.columns

X_train, y_train = np.array(bike_train[[col for col in columns if col != "rented_bike_count"]]), np.array(bike_train["rented_bike_count"])
X_test, y_test = np.array(bike_test[[col for col in columns if col != "rented_bike_count"]]), np.array(bike_test["rented_bike_count"])
X_valid, y_valid = np.array(bike_valid[[col for col in columns if col != "rented_bike_count"]]), np.array(bike_valid["rented_bike_count"])

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### MLP 1

Rete neurale con 3 hidden layers, ciascuno con 300 unità e attivazione ReLu. 

Utilizzo algorimo RMSProp come ottimizzatore. 

Utilizzo MSE come metrica. 

In [5]:
model = keras.models.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

model.compile(loss="mean_squared_error", optimizer="adam")

history = model.fit(X_train, y_train, epochs=10000, validation_data=(X_valid, y_valid), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=100)])

Epoch 1/10000
198/198 [==============================] - 2s 6ms/step - loss: 414471.0938 - val_loss: 147310.2500
Epoch 2/10000
198/198 [==============================] - 1s 5ms/step - loss: 138979.2656 - val_loss: 134273.3750
Epoch 3/10000
198/198 [==============================] - 1s 4ms/step - loss: 130979.3672 - val_loss: 129931.9141
Epoch 4/10000
198/198 [==============================] - 1s 4ms/step - loss: 126650.0234 - val_loss: 130472.6875
Epoch 5/10000
198/198 [==============================] - 1s 4ms/step - loss: 124809.6172 - val_loss: 127994.7031
Epoch 6/10000
198/198 [==============================] - 1s 4ms/step - loss: 122144.7266 - val_loss: 127594.7344
Epoch 7/10000
198/198 [==============================] - 1s 4ms/step - loss: 120943.7344 - val_loss: 126058.7812
Epoch 8/10000
198/198 [==============================] - 1s 4ms/step - loss: 119320.5391 - val_loss: 123496.4297
Epoch 9/10000
198/198 [==============================] - 1s 5ms/step - loss: 119388.8672 - val_l

Save/Load model

In [8]:
# model.save("models/bike_mlp3.h5")

In [4]:
bike_mlp3 = keras.models.load_model("models/bike_mlp3.h5")

#### RMSE

In [5]:
print_model_scores(bike_mlp3, X_train, X_valid, X_test, y_train, y_valid, y_test)

Training RMSE:  34.2898047345902
Validation RMSE:  171.26538459191516
Testing RMSE:  166.52939323970824


### MLP 2 - Dropout regularization

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation="linear")
])

model.compile(loss="mean_squared_error", optimizer="adam")

history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_valid, y_valid), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=100)])

Epoch 1/1000
198/198 [==============================] - 2s 5ms/step - loss: 439747.1875 - val_loss: 153816.7969
Epoch 2/1000
198/198 [==============================] - 1s 5ms/step - loss: 161285.7812 - val_loss: 137071.5781
Epoch 3/1000
198/198 [==============================] - 1s 5ms/step - loss: 155164.2031 - val_loss: 132142.7344
Epoch 4/1000
198/198 [==============================] - 1s 5ms/step - loss: 147204.7031 - val_loss: 130495.3203
Epoch 5/1000
198/198 [==============================] - 1s 5ms/step - loss: 147929.5312 - val_loss: 130264.0781
Epoch 6/1000
198/198 [==============================] - 1s 5ms/step - loss: 146954.1406 - val_loss: 131985.7344
Epoch 7/1000
198/198 [==============================] - 1s 5ms/step - loss: 141312.5312 - val_loss: 126409.1484
Epoch 8/1000
198/198 [==============================] - 1s 4ms/step - loss: 142024.2188 - val_loss: 126904.0938
Epoch 9/1000
198/198 [==============================] - 1s 4ms/step - loss: 141177.5312 - val_loss: 1288

Save/Load model

In [10]:
# model.save("models/bike_mlp_dropout.h5")

In [6]:
bike_mlp_dropout = keras.models.load_model("models/bike_mlp_dropout.h5")

#### RMSE

In [7]:
print_model_scores(bike_mlp_dropout, X_train, X_valid, X_test, y_train, y_valid, y_test)

Training RMSE:  95.30130549930348
Validation RMSE:  164.58012525335
Testing RMSE:  164.43951782595036


## MLP - selected predictors

Converti i dati in array numpy:

In [8]:
y_col = "rented_bike_count"
X_col = ["hour", "temperature", "humidity", "functioning_day_Yes", "seasons_Winter", 
"dew_point_temperature", "solar_radiation", "rainfall"]

X_train, y_train = np.array(bike_train[X_col]), np.array(bike_train[y_col])
X_test, y_test = np.array(bike_test[X_col]), np.array(bike_test[y_col])
X_valid, y_valid = np.array(bike_valid[X_col]), np.array(bike_valid[y_col])

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [9]:
X_train.shape

(6307, 8)

In [10]:
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

model.compile(loss="mean_squared_error", optimizer="adam")

history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_valid, y_valid), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=100)])

Epoch 1/1000
198/198 [==============================] - 2s 4ms/step - loss: 823053.4375 - val_loss: 492847.4688
Epoch 2/1000
198/198 [==============================] - 1s 3ms/step - loss: 231627.9375 - val_loss: 173871.7656
Epoch 3/1000
198/198 [==============================] - 1s 3ms/step - loss: 164981.2969 - val_loss: 162087.9375
Epoch 4/1000
198/198 [==============================] - 1s 4ms/step - loss: 159895.4375 - val_loss: 158311.0156
Epoch 5/1000
198/198 [==============================] - 1s 3ms/step - loss: 157090.9844 - val_loss: 156203.8281
Epoch 6/1000
198/198 [==============================] - 1s 3ms/step - loss: 154963.7344 - val_loss: 154020.7344
Epoch 7/1000
198/198 [==============================] - 1s 3ms/step - loss: 153246.2500 - val_loss: 152404.9062
Epoch 8/1000
198/198 [==============================] - 1s 3ms/step - loss: 150834.6719 - val_loss: 149959.7500
Epoch 9/1000
198/198 [==============================] - 1s 3ms/step - loss: 148517.7969 - val_loss: 1484

Save/Load model

In [ ]:
# model.save("models/bike_mlp4.h5")

In [11]:
bike_mlp4 = keras.models.load_model("models/bike_mlp4.h5")

#### RMSE

In [13]:
print_model_scores(bike_mlp4, X_train, X_valid, X_test, y_train, y_valid, y_test)

Training RMSE:  221.3528446030401
Validation RMSE:  246.73894666148578
Testing RMSE:  248.90999134216034
